In [4]:
import torch
import torchaudio
import matplotlib.pyplot as plt

filename = "/home/ptit/Documents/dnn-audio/Data/speech_TEST/TIMIT_TEST/DR1/FAKS0/SA1.WAV"
waveform, sample_rate = torchaudio.load(filename)

print("Shape of waveform: {}".format(waveform.size()))
print("Sample rate of waveform: {}".format(sample_rate))

#plt.figure()
#plt.plot(waveform.t().numpy())

Shape of waveform: torch.Size([1, 63488])
Sample rate of waveform: 16000
tensor([[9.1553e-05, 1.5259e-04, 6.1035e-05,  ..., 2.4414e-04, 3.0518e-04,
         2.1362e-04]])


In [3]:
######################### MEME TAILLE DE SIGNAUX
from fonctions import *

taille_sig("/home/ptit/Documents/dnn-audio/Data/speech_TEST/TIMIT_TEST/DR1")
taille_sig("/home/ptit/Documents/dnn-audio/Data/speech_TEST/TIMIT_TEST/DR2")
taille_sig("/home/ptit/Documents/dnn-audio/Data/speech_TEST/TIMIT_TEST/DR3")
taille_sig("/home/ptit/Documents/dnn-audio/Data/speech_TEST/TIMIT_TEST/DR4")
taille_sig("/home/ptit/Documents/dnn-audio/Data/speech_TEST/TIMIT_TEST/DR5")
taille_sig("/home/ptit/Documents/dnn-audio/Data/speech_TEST/TIMIT_TEST/DR6")
taille_sig("/home/ptit/Documents/dnn-audio/Data/speech_TEST/TIMIT_TEST/DR7")
taille_sig("/home/ptit/Documents/dnn-audio/Data/speech_TEST/TIMIT_TEST/DR8")

In [ ]:
######################## BRUITAGE DES SIGNAUX
import os
from fonctions import *

RSB = input('entrer le RSB voulu')
bruit, sample_rate_bruit = torchaudio.load("/Users/Ptit Miracle/Documents/projet_audio/babble.WAV")
bruit_rs = torchaudio.transforms.Resample(sample_rate_bruit, 16000)(bruit)
for subdir, dirs, files in os.walk("\\Users\\Ptit Miracle\\Documents\\projet_audio\\meme_taille_TEST"):
    for file in files:
        if ('.WAV' in file):
            subpath = os.path.join(subdir, file)
            save_bruit(subpath, bruit, bruit_rs, RSB)

C:\Users\Ptit Miracle\anaconda3\lib\site-packages\torchaudio\extension\extension.py:14: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


entrer le RSB voulu 0.7


sig_1_0.png
sig_1_1.png
sig_1_10.png
sig_1_100.png
sig_1_101.png
sig_1_102.png
sig_1_103.png
sig_1_104.png
sig_1_105.png
sig_1_106.png
sig_1_107.png
sig_1_108.png
sig_1_109.png
sig_1_11.png
sig_1_110.png
sig_1_111.png
sig_1_112.png
sig_1_113.png
sig_1_114.png
sig_1_115.png
sig_1_116.png
sig_1_12.png
sig_1_13.png
sig_1_14.png
sig_1_15.png
sig_1_16.png
sig_1_17.png
sig_1_18.png
sig_1_19.png
sig_1_2.png
sig_1_20.png
sig_1_21.png
sig_1_22.png
sig_1_23.png
sig_1_24.png
sig_1_25.png
sig_1_26.png
sig_1_27.png
sig_1_28.png
sig_1_29.png
sig_1_3.png
sig_1_30.png
sig_1_31.png
sig_1_32.png
sig_1_33.png
sig_1_34.png
sig_1_35.png
sig_1_36.png
sig_1_37.png
sig_1_38.png
sig_1_39.png
sig_1_4.png
sig_1_40.png
sig_1_41.png
sig_1_42.png
sig_1_43.png
sig_1_44.png
sig_1_45.png
sig_1_46.png
sig_1_47.png
sig_1_48.png
sig_1_49.png
sig_1_5.png
sig_1_50.png
sig_1_51.png
sig_1_52.png
sig_1_53.png
sig_1_54.png
sig_1_55.png
sig_1_56.png
sig_1_57.png
sig_1_58.png
sig_1_59.png
sig_1_6.png
sig_1_60.png
sig_1_61.png
si

In [39]:
############################### MISE EN PLACE DU CNN
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

import torch.nn as nn
import torch.optim as optim

batch_size = 512
epochs = 20
learning_rate = 1e-3

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

spec = mpimg.imread('/home/ptit/Documents/dnn-audio/Data/bruité__TEST_RSB=0.7/sig_1_0.png')

SyntaxError: invalid syntax (<ipython-input-39-d142d125d23c>, line 12)

In [37]:
class AE(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.encoder_hidden_layer = nn.Conv2d(
            in_channels=kwargs["input_shape"], out_channels=128, kernel_size=3
        )
        self.encoder_output_layer = nn.MaxPool2d(
            kernel_size=3
        )
        self.decoder_hidden_layer = nn.Conv2d(
            in_channels=64, out_channels=128, kernel_size=3
        )
        self.decoder_output_layer = nn.MaxPool2d(
            kernel_size=3
        )
        #self.decoder_output_layer = nn.MaxPool2d(
        #    in_features=128, out_features=kwargs["input_shape"]
        #)

    def forward(self, features):
        activation = self.encoder_hidden_layer(features)
        #activation = torch.relu(activation)
        code = self.encoder_output_layer(activation)
        #code = torch.sigmoid(code)
        activation = self.decoder_hidden_layer(code)
        #activation = torch.relu(activation)
        reconstructed = self.decoder_output_layer(activation)
        #reconstructed = torch.sigmoid(activation)
        return reconstructed

In [38]:
#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
model = AE(input_shape=784).to(device)

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# mean-squared error loss
criterion = nn.MSELoss()

NameError: name 'learning_rate' is not defined

In [ ]:
for epoch in range(epochs):
    loss = 0
    for batch_features, _ in train_loader:
        # reshape mini-batch data to [N, 784] matrix
        # load it to the active device
        batch_features = batch_features.view(-1, 784).to(device)
        
        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()
        
        # compute reconstructions
        outputs = model(batch_features)
        
        # compute training reconstruction loss
        train_loss = criterion(outputs, batch_features)
        
        # compute accumulated gradients
        train_loss.backward()
        
        # perform parameter update based on current gradients
        optimizer.step()
        
        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()
    
    # compute the epoch training loss
    loss = loss / len(train_loader)
    
    # display the epoch training loss
    print("epoch : {}/{}, recon loss = {:.8f}".format(epoch + 1, epochs, loss))